# INTS11

Date derived from [Tepe B, et al. (2023) Bi-allelic variants in INTS11 are associated with a complex neurological disorder. Am J Hum Genet 110:774-789. PMID:37054711](https://pubmed.ncbi.nlm.nih.gov/37054711/)

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.41


In [2]:
PMID = "PMID:37054711"
title = "Bi-allelic variants in INTS11 are associated with a complex neurological disorder"
citation = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=citation)
metadata.default_versions_with_hpo(version=hpo_version)
metadata.mondo()
print(f"HPO version {hpo_version}")

HPO version 2024-01-16


In [3]:
df = pd.read_excel("input/INTS11_Tepe_2023.xlsx")
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
df = dft
df.head()

Family,Subject,Sex,Age,Deceased,Paternal variant,Maternal variant,NM_001256456_allele_1,NM_001256456_allele_2,Global developmental delay,Intellectual disability,Language delay,Motor development and disorders,Brain MRI,Hypotonia,Seizures,Microcephaly,Optic findings,Prenatal findings,Facial dysmorphisms
Family 1,Subject 1,female,P18Y,P18Y,p.Phe45Ser,p.Arg527fs∗44,c.134T>C,c.1578_1579del,+++,+++,+++,+++,"progressive cerebral atrophy, gyral semplification, pontocerebellar atrophy",yes,yes,yes,optic atrophy,IUGR,–
Family 1.1,Subject 2,female,P6Y,P6Y,p.Phe45Ser,p.Arg527fs∗44,c.134T>C,c.1578_1579del,+++,+++,+++,+++,"progressive cerebral atrophy, gyral semplification, pontocerebellar atrophy",yes,yes,yes,optic atrophy,IUGR,–
Family 2,Subject 3,female,P5Y,NaN,p.His420Tyr,p.Leu144Phe,c.1258C>T,c.430C>T,+++,+++,+++,+,"cerebellar hypoplasia, abnormal appearance of the posterior fossa",yes,–,–,"strabismus, myopia",monozygotic twin,"hypertelorism, triangular face"
Family 2.1,Subject 4,female,P5Y,NaN,p.His420Tyr,p.Leu144Phe,c.1258C>T,c.430C>T,+++,+++,+++,+,NaN,yes,–,–,myopia,monozygotic twin,"hypertelorism, triangular face"
Family 3,Subject 5,female,P8Y,NaN,p.Val557Aspfs∗14,p.Val557Glu,c.1670_1671del,c.1670T>A,+++,+,+,+,mild to moderate cerebellar atrophy,NaN,–,–,mild optic atrophy,–,–


In [4]:
# for clarity, combine family and individual ids
def get_individual_id(row):
    family = row.name
    individual = row["Subject"]
    return f"{family}_{individual}".replace(" ", "_").replace(".","-")

df["individual_id"] = df.apply(lambda row: get_individual_id(row), axis=1)

In [5]:
#res = OptionColumnMapper.autoformat(df=df,hpo_cr=hpo_cr)
#print(res)
column_mapper_list = list()

In [6]:
gddMapper = SimpleColumnMapper(column_name="Global developmental delay",
                            hpo_id="HP:0011344",
                            hpo_label="Severe global developmental delay",
                            observed="+++", excluded="na")
column_mapper_list.append(gddMapper)
gddMapper.preview_column(df)

,mapping,count
0,"original value: ""+++"" -> HP: Severe global developmental delay (HP:0011344) (observed)",15


In [7]:
id_d = {'+++': 'Intellectual disability, severe',
                            '+':"Intellectual disability, mild"}

idMapper = OptionColumnMapper(column_name="Intellectual disability",
                                concept_recognizer=hpo_cr,
                                option_d=id_d)
column_mapper_list.append(idMapper)
idMapper.preview_column(df)

,mapping,count
0,"Intellectual disability, severe (HP:0010864) (observed)",13
1,"Intellectual disability, mild (HP:0001256) (observed)",2


In [8]:
language_delay_d = {'+++': 'Delayed speech and language development',
                    '++': 'Delayed speech and language development',
                    '+': 'Delayed speech and language development'}
excluded = {'-': 'Delayed speech and language development'}
language_delayMapper = OptionColumnMapper(column_name="Language delay", concept_recognizer=hpo_cr, option_d=language_delay_d, excluded_d=excluded)
column_mapper_list.append(language_delayMapper)
language_delayMapper.preview_column(df)

,mapping,count
0,Delayed speech and language development (HP:0000750) (observed),15


In [9]:
motor_d = {'+++': 'Motor delay','++': 'Motor delay','+': 'Motor delay',}
excluded = {'-': 'Motor delay'}
motorMapper = OptionColumnMapper(column_name="Motor development and disorders",
                                concept_recognizer=hpo_cr,
                                option_d=motor_d, excluded_d=excluded)
column_mapper_list.append(motorMapper)
motorMapper.preview_column(df)


,mapping,count
0,Motor delay (HP:0001270) (observed),15


In [10]:
brain_mri_d = {'progressive cerebral atrophy': 'Cerebral atrophy',
 'gyral semplification': 'Simplified gyral pattern',
 'pontocerebellar atrophy': 'Pontocerebellar atrophy',
 'cerebellar hypoplasia': 'Cerebellar hypoplasia',
 'mild to moderate cerebellar atrophy': 'Cerebellar atrophy',
 'subtle peripheral white matter hyperintensity around the occipital horns and to a lesser extent in the frontal regions': 'Hyperintensity of cerebral white matter on MRI',
 'simplified gyral pattern with agenesis of the corpus callosum and enlarged ventricles': ['Cerebral atrophy','Ventriculomegaly','Agenesis of corpus callosum'],
 'cerebellar progressive hypoplasia': 'Cerebellar hypoplasia',
 'progressive supratentorial and infratentorial atrophy': 'Cerebellar atrophy',
 'leukoencephalopathy': 'Leukoencephalopathy',
 '(at age 2 years) small cerebellum; delayed myelination': 'Cerebellar hypoplasia',
 '(at age 9 years) progressive cerebellar atrophy; milder pontine volume loss; diffuse white matter signal changes': 'Cerebellar atrophy',
 'cerebellar atrophy': 'Cerebellar atrophy',
 'mild brain atrophy': 'Brain atrophy',
 'mildly dilated 4th ventricle and suspected mild atrophy of the cerebellum': 'Dilated fourth ventricle',
 'periventricular signal hyperintensity and was interpreted as suspected delayed myelinization': 'Delayed CNS myelination'}
excluded = {}
brain_mriMapper = OptionColumnMapper(column_name="Brain MRI", concept_recognizer=hpo_cr, option_d=brain_mri_d, excluded_d=excluded)
column_mapper_list.append(brain_mriMapper)
brain_mriMapper.preview_column(df)


,mapping,count
0,Cerebral atrophy (HP:0002059) (observed),3
1,Simplified gyral pattern (HP:0009879) (observed),2
2,Pontocerebellar atrophy (HP:0006879) (observed),2
3,Cerebellar hypoplasia (HP:0001321) (observed),3
4,Cerebellar atrophy (HP:0001272) (observed),4
5,Hyperintensity of cerebral white matter on MRI (HP:0030890) (observed),1
6,Ventriculomegaly (HP:0002119) (observed),1
7,Agenesis of corpus callosum (HP:0001274) (observed),1
8,Leukoencephalopathy (HP:0002352) (observed),1
9,Brain atrophy (HP:0012444) (observed),1


In [11]:

hypotonia_d = {'yes': 'Hypotonia'}
excluded = {}
hypotoniaMapper = OptionColumnMapper(column_name="Hypotonia", concept_recognizer=hpo_cr, option_d=hypotonia_d, excluded_d=excluded)
column_mapper_list.append(hypotoniaMapper)
hypotoniaMapper.preview_column(df)


,mapping,count
0,Hypotonia (HP:0001252) (observed),10


In [12]:
seizuresMapper = SimpleColumnMapper(column_name="Seizures", hpo_id="HP:0001250", hpo_label="Seizure", observed="yes", excluded="no")
column_mapper_list.append(seizuresMapper)
seizuresMapper.preview_column(df)


,mapping,count
0,"original value: ""yes"" -> HP: Seizure (HP:0001250) (observed)",4
1,"original value: ""–"" -> HP: Seizure (HP:0001250) (not measured)",10
2,"original value: ""one"" -> HP: Seizure (HP:0001250) (not measured)",1


In [13]:
microcephalyMapper = SimpleColumnMapper(column_name="Microcephaly", hpo_id="HP:0000252", hpo_label="Microcephaly", observed="yes", excluded="no")
column_mapper_list.append(microcephalyMapper)
microcephalyMapper.preview_column(df)


,mapping,count
0,"original value: ""yes"" -> HP: Microcephaly (HP:0000252) (observed)",10
1,"original value: ""–"" -> HP: Microcephaly (HP:0000252) (not measured)",5


In [14]:
optic_findings_d = {'optic atrophy': 'Optic atrophy',
 'strabismus': 'Strabismus',
 'myopia': 'Myopia',
 'mild optic atrophy': 'Optic atrophy',
 'retinal dystrophy': 'Retinal dystrophy',
 'retinal dystrophy; also hypermetropia': 'Retinal dystrophy',
 'astigmatism': 'Astigmatism'}
excluded = {}
optic_findingsMapper = OptionColumnMapper(column_name="Optic findings", concept_recognizer=hpo_cr, option_d=optic_findings_d, excluded_d=excluded)
column_mapper_list.append(optic_findingsMapper)
optic_findingsMapper.preview_column(df)


,mapping,count
0,Optic atrophy (HP:0000648) (observed),5
1,Strabismus (HP:0000486) (observed),1
2,Myopia (HP:0000545) (observed),2
3,Retinal dystrophy (HP:0000556) (observed),2
4,Astigmatism (HP:0000483) (observed),1


In [15]:
prenatal_findings_d = {'IUGR': 'Intrauterine growth retardation',
 #'HELLP syndrome': 'PLACEHOLDER',
 'gestational diabetes': 'Maternal diabetes',
 'mild IUGR': 'Intrauterine growth retardation',
 'premature (33 w)': 'Premature birth',
 'microcephaly': 'Microcephaly'}
excluded = {}
prenatal_findingsMapper = OptionColumnMapper(column_name="Prenatal findings", concept_recognizer=hpo_cr, option_d=prenatal_findings_d, excluded_d=excluded)
column_mapper_list.append(prenatal_findingsMapper)
prenatal_findingsMapper.preview_column(df)


,mapping,count
0,Intrauterine growth retardation (HP:0001511) (observed),5
1,Maternal diabetes (HP:0009800) (observed),2
2,Premature birth (HP:0001622) (observed),1
3,Microcephaly (HP:0000252) (observed),1


In [16]:
facial_dysmorphisms_d = {'hypertelorism': 'Hypertelorism',
 'triangular face': 'Triangular face',
 'fontanelles closed at birth': 'Craniosynostosis',
 'low set ears': 'Low-set ears',
 'narrow palate': 'Narrow palate',
 'retrognathism': 'Retrognathia',
 #'prominent incisors': 'PLACEHOLDER',
 'coarse facies': 'Coarse facial features',
 'prominent glabella': 'Prominent glabella',
 'receding forehead': 'Sloping forehead',
 'short forehead': 'Small forehead',
 'epicanthic fold': 'Epicanthus',
 'bifid uvula': 'Bifid uvula',
 'coarse face': 'Coarse facial features',
 'broad nose and flat nasal bridge': 'Wide nose',
 'thin upper lip': 'Thin upper lip vermilion',
 'and wide mouth with macroglossia': 'Wide mouth',
 'coarse facial features': 'Coarse facial features',
 'broad nose': 'Wide nose',
 'large mouth': 'Wide mouth'}
excluded = {}
facial_dysmorphismsMapper = OptionColumnMapper(column_name="Facial dysmorphisms", concept_recognizer=hpo_cr, option_d=facial_dysmorphisms_d, excluded_d=excluded)
column_mapper_list.append(facial_dysmorphismsMapper)
facial_dysmorphismsMapper.preview_column(df)



,mapping,count
0,Hypertelorism (HP:0000316) (observed),2
1,Triangular face (HP:0000325) (observed),2
2,Craniosynostosis (HP:0001363) (observed),1
3,Low-set ears (HP:0000369) (observed),1
4,Narrow palate (HP:0000189) (observed),1
5,Retrognathia (HP:0000278) (observed),1
6,Coarse facial features (HP:0000280) (observed),4
7,Prominent glabella (HP:0002057) (observed),1
8,Sloping forehead (HP:0000340) (observed),1
9,Small forehead (HP:0000350) (observed),1


# Variants

The original publication used NM_001256456 (no version number). We use the ClinGen-preferred transcript NM_017871.6 and have checked the equivalence of the HGVS notations.

- (c.1561G>A, NM_001256456.1, p.Val515Met): c.1543G>A () 
-  (c.1258C>T, NM_001256456.1, p.His420Tyr): NM_017871.6(INTS11):c.1240C>T (p.His414Tyr) 

p.Val521Met
c.181G>A
c.1561G>Ap.Val521Met
c.181G>A
c.1561G>A

In [17]:
transcript_map_d = {
    "NM_001256456.2:c.134T>C": "NM_017871.6:c.116T>C",
"NM_001256456.2:c.1258C>T": "NM_017871.6:c.1240C>T",
"NM_001256456.2:c.1670_1671del": "NM_017871.6:c.1652_1653del",
"NM_001256456.2:c.181G>A": "NM_017871.6:c.163G>A",
"NM_001256456.2:c.671A>G": "NM_017871.6:c.653A>G",
"NM_001256456.2:c.1204A>G": "NM_017871.6:c.1186A>G",
"NM_001256456.2:c.674G>A": "NM_017871.6:c.656G>A",
"NM_001256456.2:c.1237C>T": "NM_017871.6:c.1219C>T",
"NM_001256456.2:c.667C>T": "NM_017871.6:c.649C>T",
"NM_001256456.2:c.68G>T": "NM_017871.6:c.50G>T",
"NM_001256456.2:c.1578_1579del": "NM_017871.6:c.1560_1561del",
"NM_001256456.2:c.430C>T": "NM_017871.6:c.412C>T",
"NM_001256456.2:c.1670T>A": "NM_017871.6:c.1652T>A",
"NM_001256456.2:c.1561G>A": "NM_017871.6:c.1543G>A",
"NM_001256456.2:c.1482+3G>C": "NM_017871.6:c.1464+3G>C",
"NM_001256456.2:c.1313-9G>A": "NM_017871.6:c.1295-9G>A",
"NM_001256456.2:c.954dup": "NM_017871.6:c.936dup",
"NM_001256456.2:c.52G>A": "NM_017871.6:c.34G>A",
"NM_001256456.2:c.1751A>G": "NM_017871.6:c.1733A>G"
}
def recode_variants(var):
    """code the variants from the original publication according to NM_017871.6
    Variants were checked against ClinVar and variant validator. Note we keep the transcript ids in the above
    map for clarity, but we need to remove for the variantvalidator call in VariantManager
    """
    var = f"NM_001256456.2:{var}"
    if var in transcript_map_d:
        newvar =  transcript_map_d.get(var)
        return newvar.split(":")[1]
    else:
        raise ValueError(f"Could not find {var}")

df["allele_1"] = df["NM_001256456_allele_1"].apply(lambda v: recode_variants(v))
df["allele_2"] = df["NM_001256456_allele_2"].apply(lambda v: recode_variants(v))

In [18]:
INTS11_transcript = "NM_017871.6"
vman = VariantManager(df=df, individual_column_name="individual_id", allele_1_column_name="allele_1", allele_2_column_name="allele_2", gene_symbol="INTS11", transcript=INTS11_transcript, overwrite=True)

[INFO] encoding variant "c.1560_1561del"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_017871.6%3Ac.1560_1561del/NM_017871.6?content-type=application%2Fjson
[INFO] encoding variant "c.412C>T"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_017871.6%3Ac.412C>T/NM_017871.6?content-type=application%2Fjson
[INFO] encoding variant "c.1219C>T"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_017871.6%3Ac.1219C>T/NM_017871.6?content-type=application%2Fjson
[INFO] encoding variant "c.1186A>G"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_017871.6%3Ac.1186A>G/NM_017871.6?content-type=application%2Fjson
[INFO] encoding variant "c.1652T>A"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_017871.6%3Ac.1652T>A/NM_017871.6?content-type=application%2Fjson
[INFO] encoding variant "c.653A>G"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38

In [19]:
var_d = vman.get_variant_d()

In [20]:
sexMapper = SexColumnMapper(column_name="Sex", male_symbol="male", female_symbol="female")
#sexEncoder.preview_column(df)
ageMapper = AgeColumnMapper.iso8601(column_name="Age")
#ageEncoder.preview_column(df)

In [21]:
encoder = CohortEncoder(df=df,
                        hpo_cr=hpo_cr,
                        column_mapper_list=column_mapper_list,
                        individual_column_name="individual_id",
                        agemapper=ageMapper,
                        sexmapper=sexMapper,
                        metadata=metadata)


NEDMLOB = Disease(disease_id='OMIM:620428', disease_label='Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities')
encoder.set_disease(NEDMLOB)

In [22]:
individuals = encoder.get_individuals()
from collections import defaultdict
individual_to_allele_d = defaultdict(list)
for _,row in df.iterrows():
    iid = row["individual_id"]
    allele_1 = row["allele_1"]
    allele_2 = row["allele_2"]
    var_1 = var_d.get(allele_1)
    if allele_1 == allele_2:
        var_1.set_homozygous()
        individual_to_allele_d[iid].append(var_1)
    else:
        var_2 = var_d.get(allele_2)
        var_1.set_heterozygous()
        var_2.set_heterozygous()
        individual_to_allele_d[iid].append(var_1)
        individual_to_allele_d[iid].append(var_2)
print(f"Got variants for {len(individual_to_allele_d)} individuals")


Could not parse the following as ISO8601 ages: na (n=2)
Got variants for 15 individuals


In [23]:
for i in individuals:
    allele_list = individual_to_allele_d.get(i.id)
    for a in allele_list:
        i.add_variant(a)

In [24]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.BI_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
INFORMATION,NOT_MEASURED,16


In [25]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
Family_1_Subject_1 (FEMALE; P18Y),"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities (OMIM:620428)",NM_017871.6:c.116T>C (heterozygous) NM_017871.6:c.1560_1561del (heterozygous),"Severe global developmental delay (HP:0011344); Intellectual disability, severe (HP:0010864); Delayed speech and language development (HP:0000750); Motor delay (HP:0001270); Cerebral atrophy (HP:0002059); Simplified gyral pattern (HP:0009879); Pontocerebellar atrophy (HP:0006879); Hypotonia (HP:0001252); Seizure (HP:0001250); Microcephaly (HP:0000252); Optic atrophy (HP:0000648); Intrauterine growth retardation (HP:0001511)"
Family_1-1_Subject_2 (FEMALE; P6Y),"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities (OMIM:620428)",NM_017871.6:c.116T>C (heterozygous) NM_017871.6:c.1560_1561del (heterozygous),"Severe global developmental delay (HP:0011344); Intellectual disability, severe (HP:0010864); Delayed speech and language development (HP:0000750); Motor delay (HP:0001270); Cerebral atrophy (HP:0002059); Simplified gyral pattern (HP:0009879); Pontocerebellar atrophy (HP:0006879); Hypotonia (HP:0001252); Seizure (HP:0001250); Microcephaly (HP:0000252); Optic atrophy (HP:0000648); Intrauterine growth retardation (HP:0001511)"
Family_2_Subject_3 (FEMALE; P5Y),"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities (OMIM:620428)",NM_017871.6:c.1240C>T (heterozygous) NM_017871.6:c.412C>T (heterozygous),"Severe global developmental delay (HP:0011344); Intellectual disability, severe (HP:0010864); Delayed speech and language development (HP:0000750); Motor delay (HP:0001270); Cerebellar hypoplasia (HP:0001321); Hypotonia (HP:0001252); Strabismus (HP:0000486); Myopia (HP:0000545); Hypertelorism (HP:0000316); Triangular face (HP:0000325)"
Family_2-1_Subject_4 (FEMALE; P5Y),"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities (OMIM:620428)",NM_017871.6:c.1240C>T (heterozygous) NM_017871.6:c.412C>T (heterozygous),"Severe global developmental delay (HP:0011344); Intellectual disability, severe (HP:0010864); Delayed speech and language development (HP:0000750); Motor delay (HP:0001270); Hypotonia (HP:0001252); Myopia (HP:0000545); Hypertelorism (HP:0000316); Triangular face (HP:0000325)"
Family_3_Subject_5 (FEMALE; P8Y),"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities (OMIM:620428)",NM_017871.6:c.1652_1653del (heterozygous) NM_017871.6:c.1652T>A (heterozygous),"Severe global developmental delay (HP:0011344); Intellectual disability, mild (HP:0001256); Delayed speech and language development (HP:0000750); Motor delay (HP:0001270); Cerebellar atrophy (HP:0001272); Optic atrophy (HP:0000648)"
Family_4_Subject_6 (FEMALE; P6Y),"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities (OMIM:620428)",NM_017871.6:c.163G>A (heterozygous) NM_017871.6:c.1543G>A (heterozygous),"Severe global developmental delay (HP:0011344); Intellectual disability, mild (HP:0001256); Delayed speech and language development (HP:0000750); Motor delay (HP:0001270); Hyperintensity of cerebral white matter on MRI (HP:0030890)"
Family_5_Subject_7 (FEMALE; P3Y),"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities (OMIM:620428)",NM_017871.6:c.653A>G (heterozygous) NM_017871.6:c.1464+3G>C (heterozygous),"Severe global developmental delay (HP:0011344); Intellectual disability, severe (HP:0010864); Delayed speech and language development (HP:0000750); Motor delay (HP:0001270); Cerebral atrophy (HP:0002059); Ventriculomegaly (HP:0002119); Agenesis of corpus callosum (HP:0001274); Seizure (HP:0001250); Microcephaly (HP:0000252); Craniosynostosis (HP:0001363); Low-set ears (HP:0000369)"
Family_6_Subject_8 (FEMALE; P16Y),"Neurodevelopment

In [26]:
Individual.output_individuals_as_phenopackets(individual_list=individuals, metadata=metadata)

We output 15 GA4GH phenopackets to the directory phenopackets


# HPOA 
Ages of onset were taken from the supplemental files

In [31]:
ppkt_list = [i.to_ga4gh_phenopacket(metadata) for i in individuals]
builder = HpoaTableBuilder(phenopacket_list=ppkt_list)
PMID = metadata.get_pmid()
created_by = metadata.get_created_by()
builder.neonatal_onset(PMID,8,14).infantile_onset(PMID,2,14).childhood_onset(PMID,4,14).autosomal_recessive(PMID)
hpoa_table_creator = builder.build()

We found a total of 43 unique HPO terms
Extracted disease: Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities (OMIM:620428)


In [32]:
df = hpoa_table_creator.get_dataframe()
df.head(50)

,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620428,"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities",HP:0011344,Severe global developmental delay,,,15/15,,,,,PMID:37054711,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620428,"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities",HP:0010864,"Intellectual disability, severe",,,13/13,,,,,PMID:37054711,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620428,"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities",HP:0000750,Delayed speech and language development,,,15/15,,,,,PMID:37054711,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620428,"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities",HP:0001270,Motor delay,,,15/15,,,,,PMID:37054711,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620428,"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities",HP:0002059,Cerebral atrophy,,,3/3,,,,,PMID:37054711,PCS,ORCID:0000-0002-0736-9199
5,OMIM:620428,"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities",HP:0009879,Simplified gyral pattern,,,2/2,,,,,PMID:37054711,PCS,ORCID:0000-0002-0736-9199
6,OMIM:620428,"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities",HP:0006879,Pontocerebellar atrophy,,,2/2,,,,,PMID:37054711,PCS,ORCID:0000-0002-0736-9199
7,OMIM:620428,"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities",HP:0001252,Hypotonia,,,10/10,,,,,PMID:37054711,PCS,ORCID:0000-0002-0736-9199
8,OMIM:620428,"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities",HP:0001250,Seizure,,,4/4,,,,,PMID:37054711,PCS,ORCID:0000-0002-0736-9199
9,OMIM:620428,"Neurodevelopmental disorder with motor and language delay, ocular defects, and brain abnormalities",HP:0000252,Microcephaly,,,10/10,,,,,PMID:37054711,PCS,ORCID:0000-0002-0736-9199


In [33]:
hpoa_table_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620428.tab
